In [1]:
import numpy as np
import random
import time

In [17]:
def calculate_empirical_covariance(X):
    """
    Calculates the sample covariance matrix.
    X: Data matrix (each row is an observation).
    """
    mean_est = np.mean(X, axis=0)
    cov_est = np.dot((X - mean_est).T, (X - mean_est)) / (X.shape[0])
    return cov_est

In [18]:
def estimate_shrinkage_intensity(X, empirical_cov):
    """
    Estimates the optimal shrinkage intensity.
    X: Data matrix (each row is an observation).
    sample_cov: Sample covariance matrix.
    """
    # Number of observations
    n = X.shape[0]

    # Shrinkage target (identity matrix scaled by the average variance)
    variances = np.diag(empirical_cov)
    target = np.mean(variances) * np.eye(X.shape[1])

    # Estimating the shrinkage intensity
    B = np.sum((empirical_cov - target)**2)
    Y = X - np.mean(X, axis=0)
    A = np.sum(np.array([np.outer(Y[i], Y[i]) for i in range(n)])**2) / n
    shrinkage_intensity = B / A

    # Ensure the intensity is between 0 and 1
    shrinkage_intensity = max(0, min(shrinkage_intensity, 1))
    
    return shrinkage_intensity

In [19]:
def shrinkage_covariance_matrix(X):
    """
    Computes the covariance matrix using the basic shrinkage method.
    X: Data matrix (each row is an observation).
    """
    # Empirical covariance matrix
    empirical_cov = calculate_empirical_covariance(X)

    # Estimate shrinkage intensity
    lambda_ = estimate_shrinkage_intensity(X, empirical_cov)

    # Shrinkage target (identity matrix scaled by the average variance)
    variances = np.diag(empirical_cov)
    target = np.mean(variances) * np.eye(X.shape[1])

    # Shrinkage covariance matrix
    shrinkage_cov = lambda_ * target + (1 - lambda_) * empirical_cov
    
    return shrinkage_cov

In [20]:
def ledoit_wolf_shrinkage(X):
    """
    Computes the Ledoit-Wolf shrinkage estimator for the covariance matrix.
    X: Data matrix (each row is an observation).
    """
    # Number of observations and dimensions
    n, p = X.shape

    # Empirical covariance matrix
    empirical_cov = np.cov(X, rowvar=False)

    # Mean-variance of the empirical covariance matrix
    mean_variance = np.mean(np.diag(empirical_cov))

    # Shrinkage target: scaled identity matrix
    shrinkage_target = mean_variance * np.eye(p)

    # Compute the estimator for the optimal shrinkage intensity
    X_centered = X - np.mean(X, axis=0)
    term1 = np.sum([np.outer(X_centered[i], X_centered[i]) ** 2 for i in range(n)]) / n
    term2 = np.sum((empirical_cov - shrinkage_target) ** 2)
    shrinkage_intensity = term2 / term1

    # Ensure the shrinkage intensity is between 0 and 1
    shrinkage_intensity = max(0, min(shrinkage_intensity, 1))

    # Compute the shrinkage covariance matrix
    shrinkage_cov = shrinkage_intensity * shrinkage_target + (1 - shrinkage_intensity) * empirical_cov

    return shrinkage_cov


In [21]:
def oas_shrinkage_intensity(X, empirical_cov):
    """
    Estimates the optimal shrinkage intensity using the OAS method.
    X: Data matrix (each row is an observation).
    sample_cov: Sample covariance matrix.
    """
    n, p = X.shape

    # Variance of the sample covariance matrix elements
    var_cov = np.var(empirical_cov)

    # Estimating the shrinkage intensity
    shrinkage_intensity = min((var_cov / np.square(np.trace(empirical_cov) / p)), 1.0)

    return shrinkage_intensity

def oas_covariance_matrix(X):
    """
    Computes the covariance matrix using the Oracle Approximating Shrinkage method.
    X: Data matrix (each row is an observation).
    """
    # Empirical covariance matrix
    empirical_cov = calculate_empirical_covariance(X)

    # Estimate OAS shrinkage intensity
    lambda_ = oas_shrinkage_intensity(X, empirical_cov)

    # Shrinkage target (identity matrix scaled by the average variance)
    variances = np.diag(empirical_cov)
    target = np.mean(variances) * np.eye(X.shape[1])

    # OAS covariance matrix
    oas_cov = lambda_ * target + (1 - lambda_) * empirical_cov
    
    return oas_cov


In [22]:
# Test with a random matrix
rows = random.randint(5, 10)  # Ensure at least 5 rows for better covariance estimation
cols = random.randint(5, 10)
random_matrix = np.random.randn(rows, cols)

In [23]:
# Compute the shrinkage covariance matrix
shrinkage_cov_matrix = shrinkage_covariance_matrix(random_matrix)
print("Shrinkage Covariance Matrix:")
print(shrinkage_cov_matrix)


Shrinkage Covariance Matrix:
[[ 1.27571457  0.21520941 -0.40033311 -0.33302814 -0.34875612 -0.19448405
   0.26785142]
 [ 0.21520941  0.69825881  0.14627438  0.08397125 -0.29439553 -0.35672346
  -0.21215117]
 [-0.40033311  0.14627438  0.57924264  0.08644396  0.45837959 -0.232778
  -0.11431807]
 [-0.33302814  0.08397125  0.08644396  0.60027183 -0.10314217 -0.15393458
   0.01220478]
 [-0.34875612 -0.29439553  0.45837959 -0.10314217  1.57341941 -0.57572649
  -0.19640647]
 [-0.19448405 -0.35672346 -0.232778   -0.15393458 -0.57572649  1.46634867
   0.24705126]
 [ 0.26785142 -0.21215117 -0.11431807  0.01220478 -0.19640647  0.24705126
   0.96114366]]


In [24]:
# Compute the Ledoit-Wolf shrinkage covariance matrix
ledoit_wolf_cov_matrix = ledoit_wolf_shrinkage(random_matrix)
print("Ledoit-Wolf Shrinkage Covariance Matrix:")
print(ledoit_wolf_cov_matrix)


Ledoit-Wolf Shrinkage Covariance Matrix:
[[ 1.41138041  0.23396295 -0.43521849 -0.3620485  -0.37914703 -0.21143156
   0.29119223]
 [ 0.23396295  0.78360465  0.15902086  0.09128858 -0.32004942 -0.38780865
  -0.23063821]
 [-0.43521849  0.15902086  0.6542173   0.09397676  0.49832319 -0.25306248
  -0.12427984]
 [-0.3620485   0.09128858  0.09397676  0.67707899 -0.11213007 -0.16734857
   0.01326832]
 [-0.37914703 -0.32004942  0.49832319 -0.11213007  1.7350275  -0.6258958
  -0.2135215 ]
 [-0.21143156 -0.38780865 -0.25306248 -0.16734857 -0.6258958   1.61862652
   0.26857953]
 [ 0.29119223 -0.23063821 -0.12427984  0.01326832 -0.2135215   0.26857953
   1.06939751]]


In [25]:
# Compute the OAS covariance matrix
oas_cov_matrix = oas_covariance_matrix(random_matrix)
print("Oracle Approximating Shrinkage Covariance Matrix:")
print(oas_cov_matrix)

Oracle Approximating Shrinkage Covariance Matrix:
[[ 1.23309551  0.17905032 -0.33306988 -0.27707336 -0.29015876 -0.1618072
   0.22284752]
 [ 0.17905032  0.7526628   0.12169763  0.06986256 -0.24493174 -0.29678744
  -0.17650592]
 [-0.33306988  0.12169763  0.65364351  0.0719198   0.3813635  -0.19366707
  -0.09511055]
 [-0.27707336  0.06986256  0.0719198   0.67113941 -0.08581242 -0.12807077
   0.01015416]
 [-0.29015876 -0.24493174  0.3813635  -0.08581242  1.48078053 -0.47899398
  -0.16340661]
 [-0.1618072  -0.29678744 -0.19366707 -0.12807077 -0.47899398  1.39169961
   0.20554217]
 [ 0.22284752 -0.17650592 -0.09511055  0.01015416 -0.16340661  0.20554217
   0.97137822]]


# Testing

In [27]:
true_cov = np.cov(random_matrix, rowvar=False)

In [28]:
# Function to calculate MSE
def mean_squared_error(true_cov, estimated_cov):
    return np.mean((true_cov - shrinkage_cov_matrix) ** 2)

In [29]:
basic_mse = mean_squared_error(true_cov, shrinkage_cov_matrix)

In [30]:
# Function to calculate MSE
def mean_squared_error(true_cov, estimated_cov):
    return np.mean((true_cov - ledoit_wolf_cov_matrix) ** 2)

In [31]:
ledoit_wolf_mse = mean_squared_error(true_cov, ledoit_wolf_cov_matrix)

In [32]:
# Function to calculate MSE
def mean_squared_error(true_cov, estimated_cov):
    return np.mean((true_cov - oas_cov_matrix) ** 2)

In [33]:
oas_mse = mean_squared_error(true_cov, oas_cov_matrix)

In [34]:
# Compute and time each method
start_time = time.time()
basic_cov_matrix = shrinkage_covariance_matrix(random_matrix)
basic_time = time.time() - start_time

start_time = time.time()
lw_cov_matrix = ledoit_wolf_shrinkage(random_matrix)
lw_time = time.time() - start_time

start_time = time.time()
oas_cov_matrix = oas_covariance_matrix(random_matrix)
oas_time = time.time() - start_time

In [35]:
# Print results
print("Basic Shrinkage Time: {:.4f} seconds".format(basic_time))
print("Ledoit-Wolf Time: {:.4f} seconds".format(lw_time))
print("OAS Time: {:.4f} seconds".format(oas_time))

# Print MSE 
print("Basic Shrinkage MSE: {:.4f}".format(basic_mse))
print("Ledoit-Wolf MSE: {:.4f}".format(ledoit_wolf_mse))
print("OAS MSE: {:.4f}".format(oas_mse))


Basic Shrinkage Time: 0.0003 seconds
Ledoit-Wolf Time: 0.0002 seconds
OAS Time: 0.0001 seconds
Basic Shrinkage MSE: 0.0058
Ledoit-Wolf MSE: 0.0014
OAS MSE: 0.0144
